In [1]:
### NUMBER OF DISORDERS ALIGNED


# We launch the config file with the installation of the R libraries
source("config.R")


# INPUTS : "en_product1.xml".


# OUTPUTS : "number_aligned_disorders_with_OMIM_or_OCD10"; "percentage_aligned_disorders_with_OMIM_or_OCD10";
# "number_aligned_disorders_with_OMIM"; "percentage_aligned_disorders_with_OMIM";
# "number_aligned_disorders_with_ICD10"; "percentage_aligned_disorders_with_ICD10";
# "number_OMIM_aligned_by_exact_relationship"; "percentage_OMIM_aligned_by_exact_relationship";
# "number_ICD10_aligned_by_exact_relationship"; "percentage_ICD10_aligned_by_exact_relationship".


  There is a binary version available but the source version is later:
         binary source needs_compilation
jsonlite  1.7.2  1.8.0              TRUE

  Binaries will be installed
package 'jsonlite' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'jsonlite'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\ohongnat\Anaconda3\Lib\R\library\00LOCK\jsonlite\libs\x64\jsonlite.dll vers C:\Users\ohongnat\Anaconda3\Lib\R\library\jsonlite\libs\x64\jsonlite.dll: Permission denied"Warning message:
"restored 'jsonlite'"


The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpC8gDIe\downloaded_packages

  There is a binary version available but the source version is later:
       binary  source needs_compilation
rlist 0.4.6.1 0.4.6.2             FALSE



installing the source package 'rlist'




  There is a binary version available but the source version is later:
           binary source needs_compilation
data.table 1.14.0 1.14.2              TRUE

  Binaries will be installed
package 'data.table' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'data.table'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\ohongnat\Anaconda3\Lib\R\library\00LOCK\data.table\libs\x64\datatable.dll vers C:\Users\ohongnat\Anaconda3\Lib\R\library\data.table\libs\x64\datatable.dll: Permission denied"Warning message:
"restored 'data.table'"


The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpC8gDIe\downloaded_packages
package 'XML' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'XML'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\ohongnat\Anaconda3\Lib\R\library\00LOCK\XML\libs\x64\XML.dll vers C:\Users\ohongnat\Anaconda3\Lib\R\library\XML\libs\x64\XML.dll: Permission denied"Warning message:
"restored 'XML'"


The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpC8gDIe\downloaded_packages


Warning message:
"package 'methods' is not available (for R version 3.6.1)"Warning message:
"package 'methods' is a base package, and should not be updated"

In [2]:
# We launch the libraries
library(jsonlite)
library(rlist)
library(data.table)
library(dplyr)
library(XML)
library(methods)

Warning message:
"package 'data.table' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'XML' was built under R version 3.6.3"

In [3]:
# We open xml "product1_en_cross_jdbor_evo"
product1=xmlParse("en_product1.xml")
product1=xmlToList(product1)

# We remove the last element of the list, which is the numeric count of all elements of the list
product1=product1$DisorderList
product1=product1[-length(product1)]

### Our list for working
indice_list=1
product=list()
for (i in 1:length(product1))
{
  product[[indice_list]]=product1[[i]]
  indice_list=indice_list+1
}
# Each element in the list is a disorder

In [4]:
# We only select the active clinical entities:
# Head of classification (flag=128), Historical entities (flag=512) & On-line (flag=1)
indice_list=1
all_active_clinical_entities=list()
for (i in 1:length(product))
  if (product[[i]]$DisorderFlagList$DisorderFlag$Value=='1' |
      product[[i]]$DisorderFlagList$DisorderFlag$Value=='128' |
      product[[i]]$DisorderFlagList$DisorderFlag$Value=='512')
  {
    all_active_clinical_entities[[indice_list]]=product[[i]]
    indice_list=indice_list+1
  }

In [5]:
# We only select the disorders between the active clinical entities
indice_list=1
disorders=list()
for (i in 1:length(all_active_clinical_entities))
  if (all_active_clinical_entities[[i]]$DisorderGroup$Name$text=='Disorder')
  {
    disorders[[indice_list]]=all_active_clinical_entities[[i]]
    indice_list=indice_list+1
  }
number_disorders=length(disorders)

In [6]:
# We get all the disorders that are aligned
indice_list=1
aligned_disorders=list()
for (i in 1:length(disorders))
  if (!is.null(disorders[[i]]$ExternalReferenceList$ExternalReference$.attrs))
  {
    aligned_disorders[[indice_list]]=disorders[[i]]
    indice_list=indice_list+1
  }

In [7]:
### We want the disorders aligned with at least one OMIM or ICD-10 
indice_list=1
aligned_disorders_with_OMIM_or_OCD10=list()
for (i in 1:length(aligned_disorders))
  for (j in 1:(length(aligned_disorders[[i]]$ExternalReferenceList)-1))
    if (aligned_disorders[[i]]$ExternalReferenceList[[j]]$Source=='ICD-10' |
        aligned_disorders[[i]]$ExternalReferenceList[[j]]$Source=='OMIM')
    {
      aligned_disorders_with_OMIM_or_OCD10[[indice_list]]=data.frame(aligned_disorders[[i]]$OrphaCode)
      indice_list=indice_list+1
    }
aligned_disorders_with_OMIM_or_OCD10=rbindlist(aligned_disorders_with_OMIM_or_OCD10)

# We remove duplicates
aligned_disorders_with_OMIM_or_OCD10=aligned_disorders_with_OMIM_or_OCD10[!duplicated(aligned_disorders_with_OMIM_or_OCD10)]

In [8]:
# Number of disorders with at least one OMIM or ICD-10 aligned
number_aligned_disorders_with_OMIM_or_OCD10=nrow(aligned_disorders_with_OMIM_or_OCD10)
number_aligned_disorders_with_OMIM_or_OCD10

[1] 5929

In [9]:
# Percentage of disorders with at least one OMIM or ICD-10 aligned
percentage_aligned_disorders_with_OMIM_or_OCD10=number_aligned_disorders_with_OMIM_or_OCD10/number_disorders*100
percentage_aligned_disorders_with_OMIM_or_OCD10

[1] 95.21439

In [10]:
### We want the disorders aligned with at least one OMIM 
indice_list=1
aligned_disorders_with_OMIM=list()
for (i in 1:length(aligned_disorders))
  for (j in 1:(length(aligned_disorders[[i]]$ExternalReferenceList)-1))
    if (aligned_disorders[[i]]$ExternalReferenceList[[j]]$Source=='OMIM')
    {
      aligned_disorders_with_OMIM[[indice_list]]=data.frame(aligned_disorders[[i]]$OrphaCode)
      indice_list=indice_list+1
    }
aligned_disorders_with_OMIM=rbindlist(aligned_disorders_with_OMIM)

# We remove duplicates
aligned_disorders_with_OMIM=aligned_disorders_with_OMIM[!duplicated(aligned_disorders_with_OMIM)]

In [11]:
# Number of disorders with at least one OMIM
number_aligned_disorders_with_OMIM=nrow(aligned_disorders_with_OMIM)
number_aligned_disorders_with_OMIM

[1] 3899

In [12]:
# Percentage of disorders with at least one OMIM
percentage_aligned_disorders_with_OMIM=number_aligned_disorders_with_OMIM/number_disorders*100
percentage_aligned_disorders_with_OMIM

[1] 62.61442

In [13]:
### We want the disorders aligned with at least one OMIM by an Exact relationship
indice_list=1
OMIM_aligned_by_exact_relationship=list()
for (i in 1:length(aligned_disorders))
  for (j in 1:(length(aligned_disorders[[i]]$ExternalReferenceList)-1))
    if (aligned_disorders[[i]]$ExternalReferenceList[[j]]$Source=='OMIM' &
        aligned_disorders[[i]]$ExternalReferenceList[[j]]$DisorderMappingRelation$Name$text=='E (Exact mapping: the two concepts are equivalent)')
    {
      OMIM_aligned_by_exact_relationship[[indice_list]]=data.frame(aligned_disorders[[i]]$OrphaCode)
      indice_list=indice_list+1
    }
OMIM_aligned_by_exact_relationship=rbindlist(OMIM_aligned_by_exact_relationship)

# We delete duplicates
OMIM_aligned_by_exact_relationship=OMIM_aligned_by_exact_relationship[!duplicated(OMIM_aligned_by_exact_relationship)]

In [14]:
# Number of disorders aligned with at least one OMIM by an Exact relationship
number_OMIM_aligned_by_exact_relationship=nrow(OMIM_aligned_by_exact_relationship)
number_OMIM_aligned_by_exact_relationship

[1] 3195

In [15]:
# Percentage of disorders aligned with at least one OMIM by an Exact relationship
percentage_OMIM_aligned_by_exact_relationship=number_OMIM_aligned_by_exact_relationship/number_disorders*100
percentage_OMIM_aligned_by_exact_relationship

[1] 51.30882

In [16]:
### We want the disorders aligned with at least one ICD-10
indice_list=1
aligned_disorders_with_ICD10=list()
for (i in 1:length(aligned_disorders))
  for (j in 1:(length(aligned_disorders[[i]]$ExternalReferenceList)-1))
    if (aligned_disorders[[i]]$ExternalReferenceList[[j]]$Source=='ICD-10')
    {
      aligned_disorders_with_ICD10[[indice_list]]=data.frame(aligned_disorders[[i]]$OrphaCode)
      indice_list=indice_list+1
    }
aligned_disorders_with_ICD10=rbindlist(aligned_disorders_with_ICD10)

# We remove duplicates
aligned_disorders_with_ICD10=aligned_disorders_with_ICD10[!duplicated(aligned_disorders_with_ICD10)]

In [17]:
# Number of disorders aligned with at least one ICD-10
number_aligned_disorders_with_ICD10=nrow(aligned_disorders_with_ICD10)
number_aligned_disorders_with_ICD10

[1] 5666

In [18]:
# Percentage of disorders aligned with at least one ICD-10
percentage_aligned_disorders_with_ICD10=number_aligned_disorders_with_ICD10/number_disorders*100
percentage_aligned_disorders_with_ICD10

[1] 90.99085

In [19]:
### We want the disorders aligned with at least one ICD-10 by an Exact relationship
indice_list=1
ICD10_aligned_by_exact_relationship=list()
for (i in 1:length(aligned_disorders))
  for (j in 1:(length(aligned_disorders[[i]]$ExternalReferenceList)-1))
    if (aligned_disorders[[i]]$ExternalReferenceList[[j]]$Source=='ICD-10' &
        aligned_disorders[[i]]$ExternalReferenceList[[j]]$DisorderMappingRelation$Name$text=='E (Exact mapping: the two concepts are equivalent)')
    {
      ICD10_aligned_by_exact_relationship[[indice_list]]=data.frame(aligned_disorders[[i]]$OrphaCode)
      indice_list=indice_list+1
    }
ICD10_aligned_by_exact_relationship=rbindlist(ICD10_aligned_by_exact_relationship)

# We delete duplicates
ICD10_aligned_by_exact_relationship=ICD10_aligned_by_exact_relationship[!duplicated(ICD10_aligned_by_exact_relationship)]

In [20]:
# Number of disorders aligned with at least one ICD-10 by an Exact relationship
number_ICD10_aligned_by_exact_relationship=nrow(ICD10_aligned_by_exact_relationship)
number_ICD10_aligned_by_exact_relationship

[1] 423

In [21]:
# Percentage of disorders aligned with at least one ICD-10 by an Exact relationship
percentage_ICD10_aligned_by_exact_relationship=number_ICD10_aligned_by_exact_relationship/number_disorders*100
percentage_ICD10_aligned_by_exact_relationship

[1] 6.792998